# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
 #uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['', 'Logged In', 'Kevin', 'M', '0', 'Arellano', '', 'free', 'Harrisburg-Carlisle, PA', 'GET', 'Home', '1.54001E+12', '514', '', '200', '1.54207E+12', '66']


So the line above would be skipped because it is missing the artist name. The above set of rows show us that the first value is artist, the second value is the first person and so on 

In [4]:
# check the number of rows in your csv file, the number of files have been reduced here, thus the ones with aritst that are blank are skipped
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
## Taking a glimpse of the newly created data file
df = pd.read_csv("event_datafile_new.csv")
df.head(5)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66
1,All Time Low,Maia,F,1,Burke,177.84118,free,"Houston-The Woodlands-Sugar Land, TX",510,A Party Song (The Walk of Shame),51
2,Nik & Jay,Wyatt,M,0,Scott,196.51873,free,"Eureka-Arcata-Fortuna, CA",379,Pop-Pop!,9
3,Quad City DJ's,Chloe,F,0,Cuevas,451.44771,free,"San Francisco-Oakland-Hayward, CA",506,C'mon N' Ride It (The Train) (LP Version),49
4,Hoobastank,Noah,M,1,Chavez,232.17587,free,"Ogden-Clearfield, UT",492,Born To Lead,94


# Part II. Complete the Apache Cassandra coding portion 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = {'class': 'SimpleStrategy','replication_factor' : 1}
""")

#### Set Keyspace

In [10]:
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

- Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
-  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


### Query 1
#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [11]:
session.execute("""
            CREATE TABLE IF NOT EXISTS song_session
            (sessionID INT, 
             itemInSession INT, 
             artist TEXT, 
             song_title TEXT,
             song_length FLOAT,
             PRIMARY KEY(sessionID,itemInSession))
             """)      

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## Taking only the columns that we want from each row in the excel file
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
rows = session.execute(""" SELECT sessionID, itemInSession, artist, song_title, song_length FROM song_session WHERE  sessionID  = 338 AND itemInSession = 4 """)

for row in rows:
    print(row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- Our primary key will consist of composite partition key userId, sessionId. 
- Our clustering key will be itemInSession so that our results are order by it.
- The columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.


In [14]:
session.execute("""
    CREATE TABLE IF NOT EXISTS userlib
    (userId INT, 
    sessionId INT, 
    itemInSession INT,
    artist TEXT, 
    song TEXT, 
    firstName TEXT,
    lastName TEXT, 
    PRIMARY KEY((userId, sessionId), itemInSession))
    """)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO userlib (userId, sessionId,itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Another approach to fill instead of using line[] method
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), int(itemInSession), artist, song, firstName, lastName))

In [18]:
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName FROM userlib WHERE userId = 10 AND sessionId = 182""")

for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


 ###  Query 3
 
####  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'
- Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.
- The columns of our table will be: song, firstName, lastName and userId.
- As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In [19]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("""
        CREATE TABLE IF NOT EXISTS app_history
        (song TEXT,
        firstName TEXT,
        lastName TEXT,
        userID INT,
        PRIMARY KEY(song, userID))
        """)

In [24]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history (song,firstName,lastName,userID)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],line[1],line[4],int(line[10])))

In [25]:
rows = session.execute("""SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'""")

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [26]:
session.execute("""DROP TABLE song_session""")
session.execute("""DROP TABLE userlib""")
session.execute("""DROP TABLE app_history""")

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()